In [1]:
from neo4j import GraphDatabase

import numpy as np

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "Sudoku2"))

In [3]:
create_moves_query = """
//Create moves
UNWIND RANGE(1,9) AS row
UNWIND RANGE(1,9) AS column
UNWIND RANGE(1,9) AS number
CREATE (m:Move {row:row, column:column, number:number, status:"Maybe"})
SET m.block = 3*((m.row-1)/3) + (m.column-1)/3 + 1
"""

row_requires_number_query = """
//Create row requires number constraint
UNWIND RANGE(1,9) AS row
UNWIND RANGE(1,9) AS num
CREATE (c:Constraint {row:row, number:num, type:"Row requires number."})
WITH c
MATCH (m:Move)
WHERE m.row = c.row and m.number = c.number
MERGE (m)-[:MATCHES]->(c)
"""

column_requires_number_query = """
//Create column requires number constraint
UNWIND RANGE(1,9) AS column
UNWIND RANGE(1,9) AS num
CREATE (c:Constraint {column:column, number:num, type:"Column requires number."})
WITH c
MATCH (m:Move)
WHERE m.column = c.column and m.number = c.number
MERGE (m)-[:MATCHES]->(c)
"""

block_requires_number_query = """
//Create block requires number constraint
UNWIND RANGE(1,9) AS block
UNWIND RANGE(1,9) AS num
CREATE (c:Constraint {block:block, number:num, type:"Block requires number."})
WITH c
MATCH (m:Move)
WHERE m.block = c.block and m.number = c.number
MERGE (m)-[:MATCHES]->(c)
"""

cell_requires_number_query = """
//Create cell requires number constraint
UNWIND RANGE(1,9) AS row
UNWIND RANGE(1,9) AS column
CREATE (c:Constraint {row:row, column:column, type:"Cell requires number."})
WITH c
MATCH (m:Move)
WHERE m.row = c.row and m.column = c.column
MERGE (m)-[:MATCHES]->(c)
"""

In [4]:
setup_queries = [create_moves_query, 
                 row_requires_number_query,
                 column_requires_number_query, 
                 block_requires_number_query,
                 cell_requires_number_query]

In [5]:
with driver.session() as session:
    for query in setup_queries:
        result = session.run(query)
        print(result.summary().counters)


{'labels_added': 729, 'nodes_created': 729, 'properties_set': 3645}
{'labels_added': 81, 'relationships_created': 729, 'nodes_created': 81, 'properties_set': 243}
{'labels_added': 81, 'relationships_created': 729, 'nodes_created': 81, 'properties_set': 243}
{'labels_added': 81, 'relationships_created': 729, 'nodes_created': 81, 'properties_set': 243}
{'labels_added': 81, 'relationships_created': 729, 'nodes_created': 81, 'properties_set': 243}


In [6]:
clue_grid = np.array(
[[0,4,0,0,0,0,0,2,0],
 [0,1,0,0,0,3,9,0,4],
 [8,0,7,0,0,0,0,0,0],
 [0,0,8,6,0,0,0,0,9],
 [0,3,0,0,0,0,0,8,0],
 [2,0,6,0,0,1,0,3,0],
 [0,0,0,0,1,0,0,0,0],
 [7,0,0,9,0,0,3,0,0],
 [0,0,4,0,0,6,0,7,2]
]
)

In [7]:
clue_query = "MATCH (m:Move {row:$rowNum, column:$colNum, number:$val}) SET m.status = 'Yes', m.search = 0"
it = np.nditer(clue_grid, flags=['multi_index'])
with driver.session() as session:
    for clue in it:
        if clue.item() != 0:
            result = session.run(clue_query, val=clue.item(), rowNum=it.multi_index[0] + 1, colNum=it.multi_index[1] + 1)
            print(result.summary().counters)
    result = session.run("MATCH (m:Move {status:'Yes'})-[:MATCHES*2]-(m2:Move) SET m2.status = 'No', m2.search = m.search")
    print(result.summary().counters)

{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 2}
{'properties_set': 1600}


In [8]:
find_next_search_query = """
MATCH (c:Constraint)
WHERE NOT (c)<-[:MATCHES]-(:Move {status:"Yes"})
OPTIONAL MATCH (c)<-[:MATCHES]-(m:Move {status:"Maybe"})
WITH c, m
ORDER BY id(c), id(m)
WITH c, collect(m) AS maybes
ORDER BY size(maybes), id(c)
LIMIT 1
RETURN id(maybes[0]) AS move, id(c) AS constraint
"""
def find_next_search(tx):
    result = tx.run(find_next_search_query)
    record = result.single()
    if record == None:
        return None, None
    return record['move'], record['constraint']



In [9]:
update_moves_query = """
MATCH (m:Move)
WITH max(m.search) + 1 AS newSearch
MATCH (m)-[:MATCHES]->(c:Constraint)
WHERE id(m)=$moveId and id(c)=$constraintId
SET m.status = "Yes",
m.search = newSearch,
c.search = newSearch,
c.moveId = $moveId
WITH m, newSearch
MATCH (m)-[:MATCHES]->(:Constraint)<-[:MATCHES]-(m2:Move {status:"Maybe"})
SET m2.status = "No",
m2.search = newSearch
"""
def update_moves(tx, move_id, constraint_id):
    result = tx.run(update_moves_query, moveId=move_id, constraintId=constraint_id)
    return result

In [10]:
backtrack_query = """
//Undo and search for alternate branch at last depth
MATCH (m:Move)
WHERE EXISTS(m.search)
WITH m.search AS search, collect(m) AS moves
ORDER BY m.search DESC
LIMIT 1
FOREACH(move in moves | set move.status = "Maybe")
WITH search
MATCH (c:Constraint {search:search})
OPTIONAL MATCH (c)<-[:MATCHES]-(m2 {status:"Maybe"})
WHERE (id(m2) > c.moveId OR m2 is null)
RETURN id(c) AS constraintId, id(m2) AS moveId, search
"""

def backtrack(tx):
    result = tx.run(backtrack_query)
    record = result.single()
    if record:
        return record['constraintId'], record['moveId'], record['search']
    else:
        return None, None, None

In [11]:
get_grid_query = """
//Get matrix
MATCH (m:Move {status:"Yes"})
WITH m
ORDER BY m.row, m.column
WITH m.row as rowNum, collect(m.number) as rows
RETURN COLLECT(rows) as matrix
"""
def get_grid(tx):
    result = tx.run(get_grid_query)
    return result.single()['matrix']

In [12]:
solved = False
with driver.session() as session:
    while solved == False:
        move, constraint = session.read_transaction(find_next_search)
        if move != None:
            update_result = session.write_transaction(update_moves, move, constraint)
        else:
            mc = session.run("MATCH (m:Move {status:'Maybe'}) RETURN count(*) AS maybeCount")
            maybeCount = mc.single()['maybeCount']
            if maybeCount == 0:
                solved = True
                print("Solved!")
                grid = session.read_transaction(get_grid)
                print(np.array(grid))
            else:
                constraintId, moveId, search = session.write_transaction(backtrack)
                while moveId == None:
                    session.run("MATCH (n) WHERE n.search = $search SET n.search = null, n.moveId = null", search=search)
                    constraintId, moveId, search = session.write_transaction(backtrack)
                session.run("MATCH (n) WHERE n.search = $search SET n.search = null, n.moveId = null", search=search)
                update_result = session.write_transaction(update_moves, moveId, constraintId)    

            
            

Solved!
[[3 4 9 1 6 5 7 2 8]
 [5 1 2 8 7 3 9 6 4]
 [8 6 7 2 4 9 5 1 3]
 [1 7 8 6 3 4 2 5 9]
 [4 3 5 7 9 2 6 8 1]
 [2 9 6 5 8 1 4 3 7]
 [6 2 3 4 1 7 8 9 5]
 [7 5 1 9 2 8 3 4 6]
 [9 8 4 3 5 6 1 7 2]]
